In [18]:
import glob
from music21 import converter, instrument, note, chord 

notes = []



for file in glob.glob("midi_songs/*.mid"):
    print(file)
    midi = converter.parse(file)
    notes_to_parse = None 

    parts = instrument.partitionByInstrument(midi)

    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

midi_songs/Ff7-Jenova_Absolute.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/FF8_Shuffle_or_boogie_pc.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/ahead_on_our_way_piano.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/balamb.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/DOS.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/FF3_Battle_(Piano).mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/Suteki_Da_Ne_(Piano_Version).mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/caitsith.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/sera_.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/Oppressed.mid
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
midi_songs/ff4-airship.mid
<music21.stream.iterator.RecursiveIterato

In [17]:
%pwd

'/Users/ca/code/data-bootcamp/Classical-Piano-Composer'

In [24]:
notes[0:5]

['F3', 'F3', 'B-4', 'F3', 'G4']

In [27]:
sequence_length = 100

pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i: i+sequence_length]
    sequence_out = notes[i+sequence_length ]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])



In [36]:
import numpy as np
from tensorflow.keras.utils import to_categorical

n_vocab = len(set(notes))

network_input = np.reshape(network_input, (len(network_input), sequence_length, 1))

print(n_vocab)
network_input = network_input / float(n_vocab)

network_output = to_categorical(network_output)

358


In [40]:
network_output[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')



In [47]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
892/892 [==============================] - 358s 401ms/step - loss: 4.7623
Epoch 2/200
892/892 [==============================] - 1169s 1s/step - loss: 4.7201
Epoch 3/200
892/892 [==============================] - 355s 398ms/step - loss: 4.7147
Epoch 4/200
892/892 [==============================] - 1134s 1s/step - loss: 4.7310
Epoch 5/200
152/892 [====>.........................] - ETA: 4:55 - loss: 4.6856

KeyboardInterrupt: 

In [ ]:
# test